In [1]:
import pandas as pd
import pybamm
import pybop

In [2]:
parameter_set = pybamm.ParameterValues("ECM_Example")
parameter_set.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        # "Current function [A]": 5,
        # "Initial SoC": 0.5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

In [3]:
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 2},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

file_loc = r"../hppc_lut/G1/battery_G1_cycle_2_pulse_1_lut.csv"
df = pd.read_csv(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(), 
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)

In [4]:
r_guess = 0.005
parameters = pybop.Parameters(
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r_guess, r_guess / 10),
        bounds=[0, 0.2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[1, 1500],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[0, 25000],
    ),
)

In [5]:
initial_state_of_charge= df["SoC"].iloc[0] / 100 

model.build(
    # initial_state={"Initial open-circuit voltage [V]": df["Voltage"].to_numpy()[0]}
    initial_state={"Initial SoC": initial_state_of_charge} # soc scales between 0 to 1
)
problem = pybop.FittingProblem(
    model,
    parameters,
    dataset,
)

cost = pybop.SumSquaredError(problem)

In [6]:
optim = pybop.PSO(
    cost,
    sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
    max_unchanged_iterations=30,
    max_iterations=100,
)
results = optim.run()

Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [4.84616978e-03 4.56785354e-03 4.62428495e-03 4.40112031e+02
 2.24953589e+03]
  Optimised parameters: [1.05084208e-03 2.80794596e-02 7.42131187e-03 1.85765600e+00
 9.43913668e+02]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 1.3980525980756593
  Optimisation time: 134.70644283294678 seconds
  Number of iterations: 62
  Number of evaluations: 349
  SciPy result available: No
  PyBaMM Solution available: Yes


In [7]:
pybop.plot.quick(problem, problem_inputs=results.x, title="Optimised Comparison");

In [8]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

In [1]:
import os
import pandas as pd
import pybamm
import pybop

# 298.15 K / 25 degree C
FIXED_TEMPERATURE = 298.15

# Prepare an empty list to store LUT data for each pulse
lut = []

# Loop over pulses 0 to 9
for pulse in range(10):
    file_loc = f"../hppc_lut/G1/battery_G1_cycle_2_pulse_{pulse}_lut.csv"
    print(f"Processing {file_loc}...")
    
    # Read and clean the CSV data
    df = pd.read_csv(file_loc, na_values=["NA"])
    df = df.drop_duplicates(subset=["Time"], keep="first")
    
    # Create the dataset for pybop
    dataset = pybop.Dataset({
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(), 
        "Voltage [V]": df["Voltage"].to_numpy(),
    })
    
    # Update parameter set for the ECM
    parameter_set = pybamm.ParameterValues("ECM_Example")
    parameter_set.update({
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values["Open-circuit voltage [V]"]
    })
    parameter_set.update({
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    }, check_already_exists=False)
    
    # Build the ECM model
    model = pybop.empirical.Thevenin(
        parameter_set=parameter_set,
        options={"number of rc elements": 2},
        solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
    )
    
    # Extract the initial SoC from the CSV file (scaled from 0 to 1)
    initial_soc = df["SoC"].iloc[0] / 100
    model.build(initial_state={"Initial SoC": initial_soc})
    
    # Set up the parameters for the optimization
    r_guess = 0.005
    parameters = pybop.Parameters(
        pybop.Parameter("R0 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("R1 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("R2 [Ohm]", prior=pybop.Gaussian(r_guess, r_guess / 10), bounds=[0, 0.2]),
        pybop.Parameter("C1 [F]", prior=pybop.Gaussian(500, 100), bounds=[1, 1500]),
        pybop.Parameter("C2 [F]", prior=pybop.Gaussian(2000, 500), bounds=[0, 25000]),
    )
    
    # Define and solve the fitting problem
    problem = pybop.FittingProblem(model, parameters, dataset)
    cost = pybop.SumSquaredError(problem)
    optim = pybop.PSO(
        cost,
        sigma0=[1e-3, 1e-3, 1e-3, 50, 500],
        max_unchanged_iterations=30,
        max_iterations=100,
    )
    results = optim.run()
    
    # The results vector (results.x) -- contains the optimized parameters in order to "parameters":
    # R0, R1, R2, C1, C2. We are interested in R0, R1, and C1.
    r0, r1, _, c1, _ = results.x
    
    # Create a LUT entry for the current pulse
    pulse_entry = {
        "pulse": pulse,
        "time": df["Time"].tolist(),
        "current": df["Current"].tolist(),
        "voltage": df["Voltage"].tolist(),
        "temperature": FIXED_TEMPERATURE,
        "r0": r0,
        "r1": r1,
        "c1": c1,
    }
    lut.append(pulse_entry)

# Optionally, convert the LUT list to a DataFrame and save as CSV
lut_df = pd.DataFrame(lut)
lut_df.to_csv("ecm_lut_table.csv", index=False)
print("LUT table saved to ecm_lut_table.csv")


Processing ../hppc_lut/G1/battery_G1_cycle_2_pulse_0_lut.csv...
Halt: No significant change for 30 iterations.
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [4.76734684e-03 5.35243220e-03 5.01750101e-03 4.67204515e+02
 2.57013572e+03]
  Optimised parameters: [1.74280443e-02 1.86061401e-02 9.67075019e-03 5.02626746e+00
 1.28215000e+03]
  Total-order sensitivities:
  Diagonal Fisher Information entries: None
  Final cost: 0.6431912984906181
  Optimisation time: 73.33917355537415 seconds
  Number of iterations: 52
  Number of evaluations: 317
  SciPy result available: No
  PyBaMM Solution available: Yes
Processing ../hppc_lut/G1/battery_G1_cycle_2_pulse_1_lut.csv...
